#### Notebook: Aircraft Maintenance Log Text Clasification
Deploy Custom Build ML Model as a Web Service in Azure Container Instance and Kubernetes

Version 1.0

* *Model Registration and Image Deployment:* https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-deploy-and-where
* *Azure Kubernetes Web Service:* https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice.akswebservice?view=azure-ml-py
* *Azure ML Documentation:* https://docs.microsoft.com/en-us/azure/machine-learning/service/
* *Reference Link:* https://docs.databricks.com/user-guide/libraries.html and add the below string as your **PyPi package**
* Provide this full string: **azureml-sdk[databricks]**
* You can select the option to attach the library to all clusters or just one cluster.

We have the model created in previous notebook. Now we are ready to deploy it. This notebook assumes the user subscription can create Kubernetes clusters. The major outline of this notebook is:

1. Generate small test data
2. Create a scoring file - exactly two functions - `init()` and `run()`
3. Generate docker image of the scoring file
4. Register the image.
5. Serve it as a web service (first in Container Instance and then in Kubernetes)

In [4]:
%python
logdata = sqlContext.read.parquet("dbfs:/mnt/<MY MOUNT POINT>")
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
pd_log_data = logdata.toPandas()

from sklearn.model_selection import train_test_split
Xtrain, Xtest = train_test_split(pd_log_data, test_size = 0.000005, random_state = 2) #  5 records for quick test

/databricks/python/lib/python3.6/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [5]:
%sh 
mkdir /databricks/driver/tmp

In [6]:
dbutils.fs.mount(
source = "wasbs://<container name>@<storage account name>.blob.core.windows.net",
mount_point = "/mnt/tf20",
extra_configs = {"fs.azure.account.key.aademo.blob.core.windows.net":dbutils.secrets.get(scope = "<MY SCOPE>", key = "<MY KEY>")})

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-3149201061332342> in <module> () 
 2 source = "wasbs://tf20modelassets@aademo.blob.core.windows.net" , 
 3 mount_point = "/mnt/tf20" , 
 ----> 4 extra_configs = {"fs.azure.account.key.aademo.blob.core.windows.net":dbutils.secrets.get(scope = "newscope2", key = "DbksStorageKey")})
 
 /local_disk0/tmp/1571105534159-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 306 exc . __context__ = None 
 307 exc . __cause__ = None 
 --> 308 raise exc
 309 return f_with_exception_handling
 310 

 ExecutionError : An error occurred while calling o248.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/tf20; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/tf20
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:123)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:63)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:465)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/tf20
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$insertMount$1(MetadataManager.scala:205)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:307)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:201)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:79)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:103)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:102)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:102)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:299)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:277)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$2(ServerBackend.scala:44)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:61)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:61)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$1(ServerBackend.scala:40)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$4(UsageLogging.scala:405)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:235)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:230)
	at com.databricks.logging.UsageLogging.withAttributionContext$(Us

In [7]:
%fs ls dbfs:/mnt/tf20

path,name,size
dbfs:/mnt/tf20/Xtest.npy,Xtest.npy,141384928
dbfs:/mnt/tf20/Xtrain.npy,Xtrain.npy,565536128
dbfs:/mnt/tf20/classification_report.csv,classification_report.csv,27152
dbfs:/mnt/tf20/cnn_20191013-021616_ckpt.h5,cnn_20191013-021616_ckpt.h5,408866044
dbfs:/mnt/tf20/dl_confusion.csv,dl_confusion.csv,506767
dbfs:/mnt/tf20/index2tgt.pickle,index2tgt.pickle,17492
dbfs:/mnt/tf20/index2word.pickle,index2word.pickle,3210328
dbfs:/mnt/tf20/stopword_set.pickle,stopword_set.pickle,8506
dbfs:/mnt/tf20/tgt2index.pickle,tgt2index.pickle,17492
dbfs:/mnt/tf20/word2index.pickle,word2index.pickle,3210328


In [8]:
dbutils.fs.cp('dbfs:/mnt/tf20/cnn_20191013-021616_ckpt.h5','file:/databricks/driver/tmp/cnn_20191013-021616_ckpt.h5')
dbutils.fs.cp('dbfs:/mnt/tf20/word2index.pickle','file:/databricks/driver/tmp/word2index.pickle')
dbutils.fs.cp('dbfs:/mnt/tf20/index2word.pickle','file:/databricks/driver/tmp/index2word.pickle')
dbutils.fs.cp('dbfs:/mnt/tf20/stopword_set.pickle','file:/databricks/driver/tmp/stopword_set.pickle')
dbutils.fs.cp('dbfs:/mnt/tf20/index2tgt.pickle','file:/databricks/driver/tmp/index2tgt.pickle')

Out[3]: True

In [9]:
%sh ls -lrt /databricks/driver/tmp

total 405592
-rw-r--r-- 1 root root 408866044 Oct 18 16:01 cnn_20191013-021616_ckpt.h5
-rw-r--r-- 1 root root 3210328 Oct 18 16:01 word2index.pickle
-rw-r--r-- 1 root root 3210328 Oct 18 16:01 index2word.pickle
-rw-r--r-- 1 root root 8506 Oct 18 16:01 stopword_set.pickle
-rw-r--r-- 1 root root 17492 Oct 18 16:01 index2tgt.pickle

Lets find the model and its assets.

In [11]:
SHARE_ROOT = "/databricks/driver/tmp/"
# the model in h5 format
MODEL = 'cnn_20191013-021616_ckpt.h5' # Lets use this one from today
MODEL_PATH = SHARE_ROOT + LSTM_MODEL
WORD2INDEX = 'word2index.pickle'
WORD2INDEX_PATH = SHARE_ROOT + WORD2INDEX
INDEX2WORD = 'index2word.pickle'
INDEX2WORD_PATH = SHARE_ROOT + INDEX2WORD
STOPWORD_SET = 'stopword_set.pickle'
STOPWORD_SET_PATH = SHARE_ROOT + STOPWORD_SET
INDEX2TGT = 'index2tgt.pickle'
INDEX2TGT_PATH = SHARE_ROOT + INDEX2TGT

Lets load libraries we need

In [13]:
#from keras.models import load_model
#from keras.preprocessing import sequence
import tensorflow as tf
from random import randint
import numpy as np
import pandas as pd
import nltk
import pickle
import re
import json
print(tf.__version__)

2.0.0-beta1

Lets create the two functions for scoring file: `init()` and `run()`. `init()` function is where the model and assets are loaded. run() function is where data engineering steps and scoring (predict) will be executed.

##Testing web service deployments
To test a web service deployment, you can use the run method of the Webservice object. In the following example, a JSON document is set to a web service and the result is displayed. The data sent must match what the model expects. In this example, the data format matches the input expected by the diabetes model.

In [15]:
def init():
    # read in the model file
    global model
    global word2index
    global index2word
    global stopword_set
    global index2tgt
    
    # load model
    model = tf.keras.models.load_model(MODEL_PATH)
    print("Model Loaded")
    
    # Load dictionary
    with open(WORD2INDEX_PATH, 'rb') as handle:
        word2index = pickle.load(handle)
        print("word2index dictionary loaded", type(word2index))
    # Load reverse dictionary    
    with open(INDEX2WORD_PATH, 'rb') as handle:
        index2word = pickle.load(handle)
        print("index2word dictionary loaded",type(index2word))
    # Load stopword_set    
    with open(STOPWORD_SET_PATH, 'rb') as handle:
        stopword_set = pickle.load(handle)
        print("stopword_set loaded", type(stopword_set))
    # Load index2tgt for reverse lookup of prediction
    with open(INDEX2TGT_PATH, 'rb') as handle:
        index2tgt = pickle.load(handle)
        print("index2tgt loaded", type(index2tgt))

In [16]:

def run(raw_data):
    import pandas as pd
    
    try:
        # Read json input
        data = json.loads(raw_data)['data']
        dataf = pd.read_json(data, orient='records')
        holdout_df = dataf.copy()
        
        ## parse test data to list
        interim_data_list = holdout_df['OPEN_FMR_TXT'].tolist()
        
        def replace_characters_and_stops(input_sentence, to_be_replaced, replacement_string, stops):
            # input_sentence is a sentence from a list. to_be_replaced is a string, replacement_string is the unicode replacement. 
            # first we replace special characters with space, then trim long space to one space, then convert to lower case.
            working_sentence = input_sentence.translate ({ord(c): replacement_string for c in to_be_replaced})
            fixed_trimmed = re.sub('\s+', ' ', working_sentence).strip().lower()
            filtered_sentence = ' '.join([word for word in fixed_trimmed.split() if word not in stops])
            
            return filtered_sentence
        
        cleaned_data = [] 
        for record in interim_data_list:
            fixed_record = replace_characters_and_stops(record,  '!@#$%^&*()[]{};:,./<>?\|`~-=_+', ' ', stopword_set)
            cleaned_data.append(fixed_record) 
        #print(cleaned_data)
        # Need nltk.word_tokenize to tokenize each word, then map that word to my dictionary word2index[word] to find it's value.
        num_recs = len(cleaned_data)
        MAX_SENTENCE_LENGTH = 200 # The model is trained to accept this input length.
        X = np.empty((num_recs, ), dtype=list)
        
        i = 0
        nltk.download('punkt')
        for idx, sentence in enumerate(cleaned_data):
            
            words = nltk.word_tokenize(''.join(sentence))
            
            seqs = []
            
            for word in words:
                if word in word2index:
                    seqs.append(word2index[word])
                    
                else:
                    seqs.append(word2index['UNK'])
                    
            X[i] = seqs
            i += 1
        
        X = tf.keras.preprocessing.sequence.pad_sequences(X[X !=np.array(None)].tolist(), maxlen=MAX_SENTENCE_LENGTH)
        
        # Make prediction here
        predicted_val3 = model.predict(X)
        
        # 
        value3_for_reverse_lookup = predicted_val3.argmax(axis=1)
        predicted_ata_code3 = np.vectorize(index2tgt.get)(value3_for_reverse_lookup)   
        zipped = list(zip(interim_data_list, predicted_ata_code3.tolist()))
        df = pd.DataFrame(zipped, columns = ['OPEN_FMR_TXT', 'PREDICTED_OPEN_ORIGINAL_ATA']) 
        df2json = df.to_json(orient='records')       
        return df2json

    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})

Testing `init()` to make sure it loads the model and assets properly

In [18]:
init()

Model Loaded
word2index dictionary loaded <class 'dict'>
index2word dictionary loaded <class 'dict'>
stopword_set loaded <class 'set'>
index2tgt loaded <class 'dict'>

Now we will test the `run()` function. In this step, since we want it to be a web service, we need to convert input data into JSON format, then call the `run()` function.

In [20]:
pred=run(json.dumps({"data": Xtest.to_json(orient='records')}))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data] Package punkt is already up-to-date!

Lets take a look at the prediction output. Prediction looks good. pred is json string with structure as expected. The init and run functions work properly. Tested with gpu_5p1mlbeta cluster.

In [22]:
pred

Out[24]: '[{"OPEN_FMR_TXT":"5-983NEF-C REF.2530DN *NEF* GALLEY CART DOORS\\/HANDLES\\/DIVIDERS AFT GALLEY RAIL NEEDS TO BE REATTACHED. MEL AUTHORIZED BY 317887","PREDICTED_OPEN_ORIGINAL_ATA":"2530"},{"OPEN_FMR_TXT":"A\\/C REQUIRES ETOPS PRE-DEPARTURE CK","PREDICTED_OPEN_ORIGINAL_ATA":"1210"},{"OPEN_FMR_TXT":"ETOPS PDC REQUIRED","PREDICTED_OPEN_ORIGINAL_ATA":"1210"},{"OPEN_FMR_TXT":"ZIPPER ON CARGO LINER BROKEN ON P9 6R","PREDICTED_OPEN_ORIGINAL_ATA":"5010"},{"OPEN_FMR_TXT":"6-601NEF-C REF.4420AN *NEF* PSGR AUDIO\\/VIDEO..SYSTEM\\/ZONE..... IFE SYSTEM INOP SCREEN STUCK ON INITIALIZING MEL AUTHORIZED BY 190545","PREDICTED_OPEN_ORIGINAL_ATA":"4420"}]'

#### Workspace for Hosting the Model as a Web Service
Now we are ready to designate resources for creating a web service, container image, container instance, and Kubernetes clusters.

In [24]:
#Replace this with your own subscription ID
sbscrptn = '<AZURE SUBSCRIPTION ID>'

In [25]:
from azureml.core import Workspace
ws = Workspace.create(name = 'aa_runtest_aml_workspace_demo', # or another name of your choosing
                      subscription_id=sbscrptn,
                      resource_group='tf20_deploy_rg4', # or another name of your choosing
                      create_resource_group=True,
                      location='eastus2' # or other supported Azure region
                     )
print(ws.name, "created.")

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FHGZFSXZF to authenticate.
Interactive authentication successfully completed.
UserWarning: The resource group doesn't exist or was not provided. AzureML SDK is creating a resource group=tf20_deploy_rg4 in location=eastus2 using subscription=b81e8c4c-2584-4aa7-8b10-bd1099cf015d.
Deploying KeyVault with name aarunteskeyvault10ddaf60.
Deploying StorageAccount with name aaruntesstorage377bb15e8.
Deploying AppInsights with name aaruntesinsights133a7d69.
Deployed AppInsights with name aaruntesinsights133a7d69. Took 12.69 seconds.
Deployed KeyVault with name aarunteskeyvault10ddaf60. Took 20.66 seconds.
Deployed StorageAccount with name aaruntesstorage377bb15e8. Took 33.08 seconds.
Deploying Workspace with name aa_runtest_aml_workspace_demo.
Deployed Workspace with name aa_runtest_aml_workspace_demo. Took 58.78 seconds.
aa_runtest_aml_workspace_demo created.

In [26]:
from azureml.core import Workspace

# Save the workspace config
ws.write_config()

# Reconnect to the workspace (if you're not already signed in, you'll be prompted to authenticate with a code as before)
ws = Workspace.from_config()

###Persist model assets
Next we persist the assets we have created for use in operationalization. The conda dependencies are defined in YAML file which we are going to create. This will be used to tell the webservice server which python packages are required to run this web service

In [28]:
# use azureml to create yml.
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_tensorflow_pip_package(core_type='cpu', version='2.0.0')
myenv.add_conda_package("nltk")


env_file = "env_tf20.yml"

with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

with open(env_file,"r") as f:
    print(f.read())

Saved dependency info in env_tf20.yml
# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
 # The python interpreter version.
 # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
 # Required packages for AzureML execution, history, and data preparation.
 - azureml-defaults

 - tensorflow==2.0.0
- nltk
channels:
- conda-forge

YML file needs to be in same directory as model and assets.

In [30]:
%sh cp env_tf20.yml /databricks/driver/tmp/

In [31]:
%sh cat env_tf20.yml

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
 # The python interpreter version.
 # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
 # Required packages for AzureML execution, history, and data preparation.
 - azureml-defaults

 - tensorflow==2.0.0
- nltk
channels:
- conda-forge

The score.py file is python code defining the web service operation. It includes both the `init()` and `run()` functions defined earlier imports the required libraries. These should be nearly identical to the previous defined versions.

The execution script receives data submitted to a deployed image, and passes it to the model. It then takes the response returned by the model and returns that to the client. The script is specific to your model; it must understand the data that the model expects and returns. The script usually contains two functions that load and run the model:

`init()`: Typically this function loads the model into a global object. This function is run only once when the Docker container is started.

`run(input_data)`: This function uses the model to predict a value based on the input data. Inputs and outputs to the run typically use JSON for serialization and de-serialization. You can also work with raw binary data. You can transform the data before sending to the model, or before returning to the client.

##Important:

Model loading in `score.py` `init()` function needs to use `azureml.core.Model`. Then use Keras `load_model` function. It's a two-steps process to load a model:
    ```
    
    from azureml.core.model import Model
    from keras.models import load_model
    model_path = Model.get_model_path(model_name = LSTM_MODEL)
    model = load_model(model_path)
    print("Model Loaded")```

In [33]:
%%writefile {SHARE_ROOT}score.py
import tensorflow as tf
from azureml.core.model import Model
from random import randint
import numpy as np
import pandas as pd
import nltk
import pickle
import re
import json

MODEL = 'cnn_20191013-021616_ckpt.h5'
WORD2INDEX = 'word2index.pickle'
INDEX2WORD = 'index2word.pickle'
STOPWORD_SET = 'stopword_set.pickle'
INDEX2TGT = 'index2tgt.pickle'

def init():
    # read in the model file
    global model
    global word2index
    global index2word
    global stopword_set
    global index2tgt
    
    # load model
    model_path = Model.get_model_path(model_name = MODEL)
    model = tf.keras.models.load_model(model_path)
    print("Model Loaded")
    
    # Load dictionary
    model_path = Model.get_model_path(model_name = WORD2INDEX)
    with open(model_path, 'rb') as handle:

        word2index = pickle.load(handle)
        print("word2index dictionary loaded", type(word2index))
    # Load reverse dictionary  
    model_path = Model.get_model_path(model_name = INDEX2WORD)
    with open(model_path, 'rb') as handle:
        index2word = pickle.load(handle)
        print("index2word dictionary loaded",type(index2word))
    # Load stopword_set    
    model_path = Model.get_model_path(model_name = STOPWORD_SET)
    with open(model_path, 'rb') as handle:
        stopword_set = pickle.load(handle)
        print("stopword_set loaded", type(stopword_set))
    # Load index2tgt for reverse lookup of prediction
    model_path = Model.get_model_path(model_name = INDEX2TGT)
    with open(model_path, 'rb') as handle:
        index2tgt = pickle.load(handle)
        print("index2tgt loaded", type(index2tgt))
        

def run(raw_data):
    import pandas as pd
    try:
        # Read json input
        data = json.loads(raw_data)['data']
        dataf = pd.read_json(data, orient='records')
        holdout_df = dataf.copy()
        
        ## parse test data to list
        interim_data_list = holdout_df['OPEN_FMR_TXT'].tolist()
        
        def replace_characters_and_stops(input_sentence, to_be_replaced, replacement_string, stops):
            # input_sentence is a sentence from a list. to_be_replaced is a string, replacement_string is the unicode replacement. 
            # first we replace special characters with space, then trim long space to one space, then convert to lower case.
            working_sentence = input_sentence.translate ({ord(c): replacement_string for c in to_be_replaced})
            fixed_trimmed = re.sub('\s+', ' ', working_sentence).strip().lower()
            filtered_sentence = ' '.join([word for word in fixed_trimmed.split() if word not in stops])
            
            return filtered_sentence
        
        cleaned_data = [] 
        for record in interim_data_list:
            fixed_record = replace_characters_and_stops(record,  '!@#$%^&*()[]{};:,./<>?\|`~-=_+', ' ', stopword_set)
            cleaned_data.append(fixed_record) 
        #print(cleaned_data)
        # Need nltk.word_tokenize to tokenize each word, then map that word to my dictionary word2index[word] to find it's value.
        num_recs = len(cleaned_data)
        MAX_SENTENCE_LENGTH = 200 # The model is trained to accept this input length.
        X = np.empty((num_recs, ), dtype=list)
        
        i = 0
        nltk.download('punkt')
        for idx, sentence in enumerate(cleaned_data):
            
            words = nltk.word_tokenize(''.join(sentence))
            
            seqs = []
            
            for word in words:
                if word in word2index:
                    seqs.append(word2index[word])
                    
                else:
                    seqs.append(word2index['UNK'])
                    
            X[i] = seqs
            i += 1
        
        #Zero pad each sequence at front to get sequence length MAX_SENTENCE_LENGTH
        
        X = tf.keras.preprocessing.sequence.pad_sequences(X[X !=np.array(None)].tolist(), maxlen=MAX_SENTENCE_LENGTH)
        
        # Make prediction here
        predicted_val3 = model.predict(X)
        
        # Reverse lookup back to key.
        value3_for_reverse_lookup = predicted_val3.argmax(axis=1)
        predicted_ata_code3 = np.vectorize(index2tgt.get)(value3_for_reverse_lookup)   
        zipped = list(zip(interim_data_list, predicted_ata_code3.tolist()))
        df = pd.DataFrame(zipped, columns = ['OPEN_FMR_TXT', 'PREDICTED_OPEN_ORIGINAL_ATA']) 
        df2json = df.to_json(orient='records')       
        return df2json

    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})

Writing /databricks/driver/tmp/score.py

In [34]:
%sh ls -lrt /databricks/driver/tmp

total 405604
-rw-r--r-- 1 root root 408866044 Oct 18 16:01 cnn_20191013-021616_ckpt.h5
-rw-r--r-- 1 root root 3210328 Oct 18 16:01 word2index.pickle
-rw-r--r-- 1 root root 3210328 Oct 18 16:01 index2word.pickle
-rw-r--r-- 1 root root 8506 Oct 18 16:01 stopword_set.pickle
-rw-r--r-- 1 root root 17492 Oct 18 16:01 index2tgt.pickle
-rw-r--r-- 1 root root 611 Oct 18 16:08 env_tf20.yml
-rw-r--r-- 1 root root 4582 Oct 18 16:08 score.py

####Define a Container Image
We're going to deploy the web service as a container, so we need to define a container image that includes our scoring file and denvironment dependencies.

In [36]:
%sh ls -lrt

total 28
drwxr-xr-x 2 root root 4096 Oct 18 15:50 conf
drwxr-xr-x 3 root root 4096 Oct 18 15:55 eventlogs
-rw-r--r-- 1 root root 734 Oct 18 15:55 derby.log
drwxr-xr-x 2 root root 4096 Oct 18 16:00 ganglia
drwxr-xr-x 2 root root 4096 Oct 18 16:00 logs
-rw-r--r-- 1 root root 611 Oct 18 16:08 env_tf20.yml
drwxr-xr-x 2 root root 4096 Oct 18 16:08 tmp

In [37]:
%sh cp /databricks/driver/tmp/score.py ./

In [38]:
%sh cp /databricks/driver/tmp/env_tf20.yml ./

Define a container image

In [40]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                  runtime = "python",
                                                  conda_file = "env_tf20.yml",
                                                  description = "Container image for text classification",
                                                  tags = {"data": "texts", "type": "classification"}
                                                 )
print(image_config.description, "defined.")

Container image for text classification defined.

####Define the web service deployment configuration¶
We're going to deploy the containerized web service in the Azure Container Instance (ACI) service, so we need to specify the deployment configuration.

In [42]:
from azureml.core.webservice import AciWebservice

aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 4, 
                                               tags = {"data": "texts", "type": "classification"},
                                               description = 'texts classification service')
print(aci_config.description, "defined.")

texts classification service defined.

####Deploy the web service
Now we're ready to deploy the model. The deployment process includes the following steps:

<li>Register the model file in the Azure Machine Learning service (this also uploads the local model file to your Azure Machine Learning service so it can be deployed to a container)</li>
<li>Create a container image for the web service, based on the configuration specified previously. This image will be used to instantiate the service.</li>
<li>Create a service by deploying the container image (in this case to ACI - other hosts are available!)</li>
<li>Verify the status of the deployed service.</li>
This will take some time. When deployment has completed successfully, you'll see a status of **Healthy**.

In [44]:
%sh cat /databricks/driver/.azureml/config.json

{"Id": null, "Scope": "/subscriptions/b81e8c4c-2584-4aa7-8b10-bd1099cf015d/resourceGroups/tf20_deploy_rg4/providers/Microsoft.MachineLearningServices/workspaces/aa_runtest_aml_workspace_demo"}

In [45]:
print(ws.name, ws.resource_group, ws.location, sep = '\n')

aa_runtest_aml_workspace_demo
tf20_deploy_rg4
eastus2

#### Register Model
We have to register the model and its assets.The model registry is a way to store and organize your trained models in the Azure cloud. Models are registered in your Azure Machine Learning service workspace. The model can be trained using Azure Machine Learning, or another service. In our case, we have a LSTM model, and its accompanying assets such as dictionaries and a stopword list. To register a model from file, use the following code:

In [47]:
from azureml.core.model import Model

model = Model.register(model_path = MODEL_PATH,
                       model_name = MODEL,
                       tags = {'type': "cnn", 'target': "ATA_CODE"},
                       description = "CNN model to predict ATA_CODE",
                       workspace = ws)

Registering model cnn_20191013-021616_ckpt.h5

In [48]:
word2index_model = Model.register(model_path = WORD2INDEX_PATH,
                       model_name = WORD2INDEX,
                       tags = {'type': "dict", 'target': "idx"},
                       description = "word2index dictionary for tokenize input words as tokens and map to numeric value as index",
                       workspace = ws)

Registering model word2index.pickle

In [49]:
index2word_model = Model.register(model_path =INDEX2WORD_PATH,
                       model_name = INDEX2WORD,
                       tags = {'type': "dict", 'target': "idx"},
                       description = "index2word dictionary for reverse lookup of input",
                       workspace = ws)

Registering model index2word.pickle

In [50]:
stopword_set_model = Model.register(model_path =STOPWORD_SET_PATH,
                       model_name = STOPWORD_SET,
                       tags = {'type': "set", 'target': "words"},
                       description = "stopwords list including nltk and customized",
                       workspace = ws)

Registering model stopword_set.pickle

In [51]:
index2tgt_model = Model.register(model_path =INDEX2TGT_PATH,
                       model_name = INDEX2TGT,
                       tags = {'type': "dict", 'target': "words"},
                       description = "reverse lookup dictionary for target ATA_CODE",
                       workspace = ws)

Registering model index2tgt.pickle

In [52]:
for key,value in ws.models.items():
    print("Name:", key,"\tVersion:", value.version)

Name: index2tgt.pickle Version: 1
Name: stopword_set.pickle Version: 1
Name: index2word.pickle Version: 1
Name: word2index.pickle Version: 1
Name: cnn_20191013-021616_ckpt.h5 Version: 1

#### Key to Creating Docker Image - azureml.core.image Library
Make sure `score.py` and `myenv.yml` are in same or current directory

Deployed models are packaged as an image. The image contains the dependencies needed to run the model.

For Azure Container Instance, Azure Kubernetes Service, and Azure IoT Edge deployments, the `azureml.core.image.ContainerImage` class is used to create an image configuration. The image configuration is then used to create a new Docker image.

The following code demonstrates how to create a new image configuration:

In [54]:
%sh ls -lrt

total 36
drwxr-xr-x 2 root root 4096 Oct 18 15:50 conf
drwxr-xr-x 3 root root 4096 Oct 18 15:55 eventlogs
-rw-r--r-- 1 root root 734 Oct 18 15:55 derby.log
drwxr-xr-x 2 root root 4096 Oct 18 16:00 logs
drwxr-xr-x 2 root root 4096 Oct 18 16:08 tmp
-rw-r--r-- 1 root root 4582 Oct 18 16:09 score.py
-rw-r--r-- 1 root root 611 Oct 18 16:09 env_tf20.yml
drwxr-xr-x 2 root root 4096 Oct 18 16:15 ganglia

In [55]:
import time

In [56]:
from azureml.core.image import ContainerImage
tic = time.clock()
image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                  runtime = "python",
                                                  conda_file = "env_tf20.yml",
                                                  description = "CNN model to predict ATA_CODE",
                                                  tags = {'type': "cnn", 'target': "ATA_CODE"}
                                                 )
image = ContainerImage.create(name = "ataclassification" + ".image",
                              # this is the model object
                              models = [model, word2index_model, index2word_model, stopword_set_model, index2tgt_model],
                              image_config = image_config,
                              workspace = ws)
image.wait_for_creation(show_output = True)
toc = time.clock()
print("execution time in seconds: ", toc - tic )

Creating image
Running......................................................
Succeeded
Image creation operation finished for image ataclassification.image:1, operation "Succeeded"
execution time in seconds: 1.3451129999999978

#### Key to Image Deployment - azureml.core.webservice Library
Deploy image as web service on Azure Container Instance (ACI) Note that the service creation can take few minutes.

Use Azure Container Instances for deploying your models as a web service if one or more of the following conditions is true:

1. You need to quickly deploy and validate your model.
2. You are testing a model that is under development. 

| Method    | Note |
| ----------- | ----------- |
| `deploy_from_image`      | You must register the model and create an image before using this method.       |
| `deploy`   | When using this method, you do not need to register the model or create the image. However you cannot control the name of the model or image, or associated tags and descriptions.        |
| `deploy_from_model`   | When using this method, you do not need to create an image. But you do not have control over the name of the image that is created.        |

This example uses deploy_from_image.

In [58]:
# Azure Container Service (ACI) Name
ACI_SERVICE_NAME = "atacode" + '-aciservice'

# Azure Kubernetes Service (AKS) Name
AKS_SERVICE_NAME = "atacode" + '-aksservice'

In [59]:
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice

aci_service_name = ACI_SERVICE_NAME.lower()

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 4, 
                                               tags = {'type': "cnn", 'target': "ATA_CODE"}, 
                                               description = "CNN model to predict ATA_CODE")

In [60]:
#Let's see if we have an ACI web service already running in the workspace
aci_service = ""
for aci in AciWebservice.list(workspace=ws):
    if (aci.compute_type == "ACI"):
        if (aci.name == aci_service_name): 
            aci_service = aci
            print("Existing ACI Service name:", aci_service.name)
        else:
            print("No service by the name of **"+aci_service_name+"** exists!")

In [61]:
%%time
# We update the image if service exists or create a new service if doesnt exist
if (aci_service == ""):
    aci_service = AciWebservice.deploy_from_image(deployment_config = aciconfig,
                                           image = image,
                                           name = aci_service_name,
                                           workspace = ws)
    aci_service.wait_for_deployment(True)
    print(aci_service.state)
else:
    aci_service.update(image=image)
    aci_service.wait_for_deployment(True)
    print(aci_service.state)

Running.............................
SucceededACI service creation operation finished, operation "Succeeded"
Healthy
CPU times: user 477 ms, sys: 182 ms, total: 659 ms
Wall time: 2min 29s

In [62]:
print(aci_service.get_logs())

2019-10-18T16:28:23,431156716+00:00 - gunicorn/run 
2019-10-18T16:28:23,431156716+00:00 - rsyslog/run 
2019-10-18T16:28:23,431854716+00:00 - iot-server/run 
2019-10-18T16:28:23,695078917+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2019-10-18T16:28:24,344527166+00:00 - iot-server/finish 1 0
2019-10-18T16:28:24,354978170+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 19.9.0
Listening at: http://127.0.0.1:31311 (12)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 47
Initializing logger
Starting up app insights client
Starting up request id generator
Starting up app insight hooks
Invoking user's init function
2019-10-18 16:28:36,025 | azureml.core.run | DEBUG | Could not load run context RunEnvironmentException:
	Message: Could not load a submitted run, if outside of an execution context, use experiment.start_logging to initialize an azureml.core.Run.
	InnerException None
	ErrorResponse 
{
 "error": {
 "message": "Could not load a submitted run, if outside of an execution context, use experiment.start_logging to initialize an azureml.core.Run."
 }
}, switching offline: False
2019-10-18 16:28:36,025 | azureml.core.run | DEBUG | Could not load the run context and allow_offline set to False
2019-10-18 16:28:36,025 | azureml.core.model | DEBUG | RunEnvironmentException: RunEnvironmentException:
	Message: Could not load a submitted run, if outside of an execution context, use experiment.start_logging to initialize an azureml.core.Run.
	InnerException RunEnvironmentException:
	Message: Could not load a submitted run, if outside of an execution context, use experiment.start_logging to initialize an azureml.core.Run.
	InnerException None
	ErrorResponse 
{
 "error": {
 "message": "Could not load a submitted run, if outside of an execution context, use experiment.start_logging to initialize an azureml.core.Run."
 }
}
	ErrorResponse 
{
 "error": {
 "message": "Could not load a submitted run, if outside of an execution context, use experiment.start_logging to initialize an azureml.core.Run."
 }
}
2019-10-18 16:28:36,025 | azureml.core.model | DEBUG | version is None. Latest version is 1
2019-10-18 16:28:36,026 | azureml.core.model | DEBUG | Found model path at azureml-models/cnn_20191013-021616_ckpt.h5/1/cnn_20191013-021616_ckpt.h5
2019-10-18 16:28:36.053078: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-10-18 16:28:36.067177: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2294685000 Hz
2019-10-18 16:28:36.067652: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x557bc1fc8cb0 executing computations on platform Host. Devices:
2019-10-18 16:28:36.067758: I tensorflow/compiler/xla/service/service.cc:175] StreamExecutor device (0): Host, Default Version
Model Loaded
2019-10-18 16:28:38,325 | azureml.core.run | DEBUG | Could not load run context RunEnvironmentException:
	Message: Could not load a submitted run, if outside of an execution context, use experiment.start_logging to initialize an azureml.core.Run.
	InnerException None
	ErrorResponse 
{
 "error": {
 "message": "Could not load a submitted run, if outside of an execution context, use experiment.start_logging to initialize an azureml.core.Run."
 }
}, switching offline: False
2019-10-18 16:28:38,325 | azureml.core.run | DEBUG | Could not load the run context and allow_offline set to False
2019-10-18 16:28:38,325 | azureml.core.model | DEBUG | RunEnvironmentException: RunEnvironmentException:
	Message: Could not load a submitted run, if outside of an execution context, use experiment.start_logging to initialize an azureml.core.Run.
	InnerException RunEnvironmentException:
	Message: Could not load a submitted run, if outside of an execution context, use experiment.start_logging to initialize an azureml.core.Run.
	InnerException None
	ErrorResponse 

In [63]:
print(aci_service.name)
print(aci_service.state)
print(aci_service.location)
print(aci_service.image_id)
print(aci_service.scoring_uri)
print(aci_service.description)
print(aci_service.tags)

atacode-aciservice
Healthy
eastus2
ataclassification.image:1
http://3b568015-5459-4ee8-a943-1a1a1b4e99b6.eastus2.azurecontainer.io/score
CNN model to predict ATA_CODE
{'type': 'cnn', 'target': 'ATA_CODE'}

In [64]:
print('web service hosted in ACI:', aci_service.scoring_uri)

web service hosted in ACI: http://3b568015-5459-4ee8-a943-1a1a1b4e99b6.eastus2.azurecontainer.io/score

In [65]:
test_sample = json.dumps({"data": Xtest.to_json(orient='records')})

In [66]:
prediction = aci_service.run(input_data = test_sample)
print(prediction)

[{"OPEN_FMR_TXT":"5-983NEF-C REF.2530DN *NEF* GALLEY CART DOORS\/HANDLES\/DIVIDERS AFT GALLEY RAIL NEEDS TO BE REATTACHED. MEL AUTHORIZED BY 317887","PREDICTED_OPEN_ORIGINAL_ATA":"2530"},{"OPEN_FMR_TXT":"A\/C REQUIRES ETOPS PRE-DEPARTURE CK","PREDICTED_OPEN_ORIGINAL_ATA":"1210"},{"OPEN_FMR_TXT":"ETOPS PDC REQUIRED","PREDICTED_OPEN_ORIGINAL_ATA":"1210"},{"OPEN_FMR_TXT":"ZIPPER ON CARGO LINER BROKEN ON P9 6R","PREDICTED_OPEN_ORIGINAL_ATA":"5010"},{"OPEN_FMR_TXT":"6-601NEF-C REF.4420AN *NEF* PSGR AUDIO\/VIDEO..SYSTEM\/ZONE..... IFE SYSTEM INOP SCREEN STUCK ON INITIALIZING MEL AUTHORIZED BY 190545","PREDICTED_OPEN_ORIGINAL_ATA":"4420"}]

#### Key to Deploying image as web service on Azure Kubernetes (AKS) - azureml.core.webservice and azureml.core.compute Libraries

To deploy your model as a high-scale production web service, use Azure Kubernetes Service (AKS). You can use an existing AKS cluster or create a new one using the Azure Machine Learning SDK, CLI, or the Azure portal.

Creating an AKS cluster is a one time process for your workspace. You can reuse this cluster for multiple deployments. If you delete the cluster, then you must create a new cluster the next time you need to deploy.

Azure Kubernetes Service provides the following capabilities:

1. Autoscaling
2. Logging
3. Model data collection
4. Fast response times for your web services
5. TLS termination
6. Authentication

In [68]:
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import AksWebservice

In [69]:
#Let's see first if we have a compute cluster available in the workspace
aks_name = 'atacode-aks'
aks_target = ""
aks_service_name = AKS_SERVICE_NAME.lower() 
aks_service = ""

for aks in AksCompute.list(ws):
    if (aks.name == aks_name): 
        aks_target = aks
        print("Existing Cluster name: ", aks_target.name)

for akss in AksWebservice.list(ws):
      if (akss.name == aks_service_name): 
        aks_service = akss
        print("Existing AKS Web Service name: ", aks_service.name)

In [70]:
%%time
if (aks_service == ""):
    # Set the AKS Cluster configuration
    prov_config = AksCompute.provisioning_configuration(agent_count=6, vm_size="Standard_DS2_v2", ssl_cname=None, 
                                                        ssl_cert_pem_file=None, ssl_key_pem_file=None, 
                                                        location="EastUs2")

    # Create the cluster
    aks_target = ComputeTarget.create(workspace = ws, 
                                     name = aks_name, 
                                     provisioning_configuration = prov_config)

    aks_target.wait_for_completion(show_output = True)
    print(aks_target.provisioning_errors)

    #Set the web service configuration
    aks_config = AksWebservice.deploy_configuration(autoscale_enabled=True, autoscale_min_replicas=3, 
                                                autoscale_max_replicas=10, autoscale_refresh_seconds=None, 
                                                autoscale_target_utilization=80, collect_model_data=None, 
                                                cpu_cores=None, memory_gb=None, enable_app_insights=True, 
                                                scoring_timeout_ms=None, replica_max_concurrent_requests=None, 
                                                num_replicas=None, primary_key=None, secondary_key=None, 
                                                tags = {'type': "cnn", 'target': "ATA_CODE"}, 
                                                description="AKS Service")

    # Create the Web Service
    aks_service = AksWebservice.deploy_from_image(workspace = ws, 
                                           name = aks_service_name,
                                           image = image,
                                           deployment_config = aks_config,
                                           deployment_target = aks_target)
    
    aks_service.wait_for_deployment(show_output = True)
    print(aks_service.state)
    
else:
    aks_service.update(image=image)
    aks_service.wait_for_deployment(show_output = True)
    print(aks_service.state)

Creating.................................................................................................................................................................................................
SucceededProvisioning operation finished, operation "Succeeded"
None
Running................
SucceededAKS service creation operation finished, operation "Succeeded"
Healthy
CPU times: user 3.44 s, sys: 1.08 s, total: 4.52 s
Wall time: 18min 25s

In [71]:
print("Name:", aks_target.name)
print("Agent Count:", aks_target.agent_count)
print("VM Size:", aks_target.agent_vm_size)
print("Location:", aks_target.location)

Name: atacode-aks
Agent Count: 6
VM Size: STANDARD_DS2_V2
Location: eastus2

In [72]:
print(aks_service.name)
print(aks_service.state)
print(aks_service.image_id)
print(aks_service.scoring_uri)
print(aks_service.get_keys()[0])

atacode-aksservice
Healthy
ataclassification.image:1
http://137.116.85.79:80/api/v1/service/atacode-aksservice/score
7AzZnS1imhCHjnsahsHO9woeFrYm1mlh

In [73]:
print(aks_service.get_keys())

('7AzZnS1imhCHjnsahsHO9woeFrYm1mlh', 'oh1by1Is1h7L7hpD0WrIvMPklAUwu09c')

In [74]:
aks_service

Out[63]: AksWebservice(workspace=Workspace.create(name='aa_runtest_aml_workspace_demo', subscription_id='b81e8c4c-2584-4aa7-8b10-bd1099cf015d', resource_group='tf20_deploy_rg4'), name=atacode-aksservice, image_id=ataclassification.image:1, compute_type=AKS, state=Healthy, scoring_uri=http://137.116.85.79:80/api/v1/service/atacode-aksservice/score, tags={'type': 'cnn', 'target': 'ATA_CODE'}, properties={})

In [75]:
print('web service hosted in AKS:', aks_service.scoring_uri)

web service hosted in AKS: http://137.116.85.79:80/api/v1/service/atacode-aksservice/score

In [76]:
test_sample

Out[65]: '{"data": "[{\\"LOG_ID\\":\\"878-34701643-20180521\\",\\"OPEN_DATE\\":1526860800000,\\"FLEET\\":\\"A320\\",\\"AIRLINE\\":\\"AA\\",\\"NOSE\\":\\"878\\",\\"OPEN_FMR_TXT\\":\\"5-983NEF-C REF.2530DN *NEF* GALLEY CART DOORS\\\\/HANDLES\\\\/DIVIDERS AFT GALLEY RAIL NEEDS TO BE REATTACHED. MEL AUTHORIZED BY 317887\\",\\"OPEN_ORIGINAL_ATA\\":\\"2530\\",\\"__index_level_0__\\":937801},{\\"LOG_ID\\":\\"861-34683012-20180424\\",\\"OPEN_DATE\\":1524528000000,\\"FLEET\\":\\"A320\\",\\"AIRLINE\\":\\"AA\\",\\"NOSE\\":\\"861\\",\\"OPEN_FMR_TXT\\":\\"A\\\\/C REQUIRES ETOPS PRE-DEPARTURE CK\\",\\"OPEN_ORIGINAL_ATA\\":\\"1210\\",\\"__index_level_0__\\":918120},{\\"LOG_ID\\":\\"7BS-35517002-20180509\\",\\"OPEN_DATE\\":1525824000000,\\"FLEET\\":\\"B777\\",\\"AIRLINE\\":\\"AA\\",\\"NOSE\\":\\"7BS\\",\\"OPEN_FMR_TXT\\":\\"ETOPS PDC REQUIRED\\",\\"OPEN_ORIGINAL_ATA\\":\\"1210\\",\\"__index_level_0__\\":788529},{\\"LOG_ID\\":\\"390-33945100-20180701\\",\\"OPEN_DATE\\":1530403200000,\\"FLEET\\":\\"B767\\",\\"AIRLINE\\":\\"AA\\",\\"NOSE\\":\\"390\\",\\"OPEN_FMR_TXT\\":\\"ZIPPER ON CARGO LINER BROKEN ON P9 6R\\",\\"OPEN_ORIGINAL_ATA\\":\\"5010\\",\\"__index_level_0__\\":203513},{\\"LOG_ID\\":\\"3LL-34713941-20180606\\",\\"OPEN_DATE\\":1528243200000,\\"FLEET\\":\\"B737\\",\\"AIRLINE\\":\\"AA\\",\\"NOSE\\":\\"3LL\\",\\"OPEN_FMR_TXT\\":\\"6-601NEF-C REF.4420AN *NEF* PSGR AUDIO\\\\/VIDEO..SYSTEM\\\\/ZONE..... IFE SYSTEM INOP SCREEN STUCK ON INITIALIZING MEL AUTHORIZED BY 190545\\",\\"OPEN_ORIGINAL_ATA\\":\\"4420\\",\\"__index_level_0__\\":399489}]"}'

In [77]:
Xtest.to_json(orient='records')

Out[66]: '[{"LOG_ID":"878-34701643-20180521","OPEN_DATE":1526860800000,"FLEET":"A320","AIRLINE":"AA","NOSE":"878","OPEN_FMR_TXT":"5-983NEF-C REF.2530DN *NEF* GALLEY CART DOORS\\/HANDLES\\/DIVIDERS AFT GALLEY RAIL NEEDS TO BE REATTACHED. MEL AUTHORIZED BY 317887","OPEN_ORIGINAL_ATA":"2530","__index_level_0__":937801},{"LOG_ID":"861-34683012-20180424","OPEN_DATE":1524528000000,"FLEET":"A320","AIRLINE":"AA","NOSE":"861","OPEN_FMR_TXT":"A\\/C REQUIRES ETOPS PRE-DEPARTURE CK","OPEN_ORIGINAL_ATA":"1210","__index_level_0__":918120},{"LOG_ID":"7BS-35517002-20180509","OPEN_DATE":1525824000000,"FLEET":"B777","AIRLINE":"AA","NOSE":"7BS","OPEN_FMR_TXT":"ETOPS PDC REQUIRED","OPEN_ORIGINAL_ATA":"1210","__index_level_0__":788529},{"LOG_ID":"390-33945100-20180701","OPEN_DATE":1530403200000,"FLEET":"B767","AIRLINE":"AA","NOSE":"390","OPEN_FMR_TXT":"ZIPPER ON CARGO LINER BROKEN ON P9 6R","OPEN_ORIGINAL_ATA":"5010","__index_level_0__":203513},{"LOG_ID":"3LL-34713941-20180606","OPEN_DATE":1528243200000,"FLEET":"B737","AIRLINE":"AA","NOSE":"3LL","OPEN_FMR_TXT":"6-601NEF-C REF.4420AN *NEF* PSGR AUDIO\\/VIDEO..SYSTEM\\/ZONE..... IFE SYSTEM INOP SCREEN STUCK ON INITIALIZING MEL AUTHORIZED BY 190545","OPEN_ORIGINAL_ATA":"4420","__index_level_0__":399489}]'

In [78]:
test_sample = json.dumps({"data": Xtest.to_json(orient='records')})

prediction = aks_service.run(input_data = test_sample)
print(prediction)

[{"OPEN_FMR_TXT":"5-983NEF-C REF.2530DN *NEF* GALLEY CART DOORS\/HANDLES\/DIVIDERS AFT GALLEY RAIL NEEDS TO BE REATTACHED. MEL AUTHORIZED BY 317887","PREDICTED_OPEN_ORIGINAL_ATA":"2530"},{"OPEN_FMR_TXT":"A\/C REQUIRES ETOPS PRE-DEPARTURE CK","PREDICTED_OPEN_ORIGINAL_ATA":"1210"},{"OPEN_FMR_TXT":"ETOPS PDC REQUIRED","PREDICTED_OPEN_ORIGINAL_ATA":"1210"},{"OPEN_FMR_TXT":"ZIPPER ON CARGO LINER BROKEN ON P9 6R","PREDICTED_OPEN_ORIGINAL_ATA":"5010"},{"OPEN_FMR_TXT":"6-601NEF-C REF.4420AN *NEF* PSGR AUDIO\/VIDEO..SYSTEM\/ZONE..... IFE SYSTEM INOP SCREEN STUCK ON INITIALIZING MEL AUTHORIZED BY 190545","PREDICTED_OPEN_ORIGINAL_ATA":"4420"}]

The model image successfully ran on Kubernetes cluster. 

### What we have accomplished:
1. Register a custom-built machine learning model and its assets.
2. Create an image of the model and its assets.
3. Test the image deployment in ACI and Kubernetes using JSON input.

#### Clean up

To delete a deployed web service, use `service.delete()`.

To delete an image, use `image.delete()`.

To delete a registered model, use `model.delete()`.

In [81]:
%%time
aci_service.delete()
aks_service.delete()
aks_target.delete()
image.delete()
model.delete()
word2index_model.delete()
index2word_model.delete()
stopword_set_model.delete()
index2tgt_model.delete()


CPU times: user 155 ms, sys: 21 ms, total: 176 ms
Wall time: 11.8 s

In [82]:
print(aks_service.state)
print(aci_service.state)

Deleting
Deleting